In [1]:
import time
import numpy as np
from tqdm.notebook import tqdm

from components.dataset_jo import *

dataset = Dataset_subjectDependent('data')
segment_lenght = 60 #second
dataset.set_segment(7680//(128*segment_lenght))
for filename in dataset.get_file_list():
    data, labels, _ = dataset.get_data(filename, return_type='numpy')
    print(filename, labels.shape, labels.sum(axis=0)/labels.shape[0]*100)
    break

Found: 32 files
s01 (40, 2) [47.5 60. ]


In [2]:
def train_model(X_ori,y_ori,groups_ori,filename=None, kernel='rbf',return_text=False):
    # Make a copy because I am paranoid
    X,y,groups = X_ori.copy(), y_ori.copy(), groups_ori.copy()

    from sklearn.svm import SVC
    from sklearn.model_selection import GroupShuffleSplit 
    from sklearn.model_selection import cross_val_score

    model = SVC(kernel=kernel,max_iter=50000)
    cv = GroupShuffleSplit(n_splits=10, train_size=0.75, random_state=0)
    cross = cross_val_score(model, X, y, cv=cv,groups=groups, n_jobs=8)
    
    # We probably dont need this
    model = SVC(kernel=kernel, max_iter=50000)
    model.fit(X, y)
    ans = model.predict(X)
    acc = sum(ans == y) / len(y)
    # If the model answer with all 0 or 1, we print this message
    text = None
    if( sum(ans) == len(y) or sum(ans) == 0 ): 
        text = f"-----WARNING: Model {filename} failed to learn: sum(ans)={sum(ans)} sum(y)={sum(y)} len(y)={len(y)}"
    if(return_text):
        return model, acc, cross, text
    else:
        if(text != None): print(text)
        return model, acc, cross

## 2. Spectral Features

In [3]:
from mne_features.feature_extraction import FeatureExtractor

In [4]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})


accs,cv_means,cv_stds  = [],[],[]
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

accs_v,cv_means_v,cv_stds_v  = [],[],[]
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

-----WARNING: Model s01 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	AROUSAL-s01|Acc=0.6|10-CV=0.56|STD=0.1562|Time spend=34.281423568725586
-----WARNING: Model s02 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	AROUSAL-s02|Acc=0.6|10-CV=0.54|STD=0.11136|Time spend=0.48838353157043457
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=8.0 len(y)=40
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=0.49751853942871094
-----WARNING: Model s04 failed to learn: sum(ans)=0.0 sum(y)=16.0 len(y)=40
	AROUSAL-s04|Acc=0.6|10-CV=0.53|STD=0.12689|Time spend=0.5138452053070068
-----WARNING: Model s05 failed to learn: sum(ans)=0.0 sum(y)=19.0 len(y)=40
	AROUSAL-s05|Acc=0.525|10-CV=0.37|STD=0.09|Time spend=0.5337028503417969
	AROUSAL-s06|Acc=0.65|10-CV=0.53|STD=0.14866|Time spend=0.5668420791625977
-----WARNING: Model s07 failed to learn: sum(ans)=40.0 sum(y)=25.0 len(y)=40
	AROUSAL-s07|Acc=0.625|10-CV=0.65|STD=0.14318|Time spend=0.5326733589172363
-----WARNING: Model s

## 3. Asymetry

In [5]:
channels = ['Fp1','AF3','F3','F7','FC5','FC1','C3','T7','CP5','CP1','P3','P7','PO3','O1','Oz','Pz','Fp2','AF4','Fz','F4','F8','FC6','FC2','Cz','C4','T8','CP6','CP2','P4','P8','PO4','O2']
left_channels = ['Fp1','F7','F3','T7','P7','C3','P3','O1','AF3','FC5','FC1','CP5','CP1','PO3']
right_channels = ['Fp2','F8','F4','T8','P8','C4','P4','O2','AF4','FC6','FC2','CP6','CP2','PO4']
left_channel_indexes = [ channels.index(ch) for ch in left_channels ]
right_channel_indexes = [ channels.index(ch) for ch in right_channels ]

print(f"{left_channel_indexes=}")
print(f"{right_channel_indexes=}")

frontal_channels = ['FC5','FC1','FC2','FC6','F7','F3','Fz','F4','F8','Fp1','Fp2']
posterior_channels = ['CP5','CP1','CP2','CP6','P7','P3','Pz','P4','P8','O1','O2']

frontal_channel_indexes = [ channels.index(ch) for ch in frontal_channels ]
posterior_channel_indexes = [ channels.index(ch) for ch in posterior_channels ]

print(f"{frontal_channel_indexes=}")
print(f"{posterior_channel_indexes=}")

left_channel_indexes=[0, 3, 2, 7, 11, 6, 10, 13, 1, 4, 5, 8, 9, 12]
right_channel_indexes=[16, 20, 19, 25, 29, 24, 28, 31, 17, 21, 22, 26, 27, 30]
frontal_channel_indexes=[4, 5, 22, 21, 3, 2, 18, 19, 20, 0, 16]
posterior_channel_indexes=[8, 9, 27, 26, 11, 10, 15, 28, 29, 13, 31]


In [6]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

accs,cv_means,cv_stds  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1), groups, filename=filename)
        print(f"\tAROUSAL-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs[feature_name].append(acc)
        cv_means[feature_name].append(cross.mean())
        cv_stds[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"AROUSAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

accs_v,cv_means_v,cv_stds_v  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1), groups, filename=filename)
        print(f"\tVALENCE-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs_v[feature_name].append(acc)
        cv_means_v[feature_name].append(cross.mean())
        cv_stds_v[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"VALENCE-{feature_name}|Acc={sum(accs_v[feature_name])/len(accs_v[feature_name])}|10-CV={sum(cv_means_v[feature_name])/len(cv_means_v[feature_name])}|STD={sum(cv_stds_v[feature_name])/len(cv_stds_v[feature_name])}")

for feature_name in ['DASM','RASM','DCAU']:
    accs[feature_name].extend(accs_v[feature_name])
    cv_means[feature_name].extend(cv_means_v[feature_name])
    cv_stds[feature_name].extend(cv_stds_v[feature_name])
    print(f"TOTAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

-----WARNING: Model s01 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	AROUSAL-s01-DASM|Acc=0.6|10-CV=0.56|STD=0.1562|Time spend=0.26605939865112305
	AROUSAL-s01-RASM|Acc=0.625|10-CV=0.5|STD=0.16733|Time spend=0.2762484550476074
-----WARNING: Model s01 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	AROUSAL-s01-DCAU|Acc=0.6|10-CV=0.56|STD=0.1562|Time spend=0.2852480411529541
-----WARNING: Model s02 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	AROUSAL-s02-DASM|Acc=0.6|10-CV=0.57|STD=0.13454|Time spend=0.24341893196105957
-----WARNING: Model s02 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	AROUSAL-s02-RASM|Acc=0.6|10-CV=0.57|STD=0.13454|Time spend=0.25293993949890137
-----WARNING: Model s02 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	AROUSAL-s02-DCAU|Acc=0.6|10-CV=0.54|STD=0.11136|Time spend=0.2611348628997803
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=8.0 len(y)=40
	AROUSAL-s03-DASM|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=0.23

## 4. Connectivity

### 4.1 $ \text{PCC}_{\text{time}}(i,j) = \frac{\text{Cov}[\mathbf{X}_i, \mathbf{X}_j]}{\sigma_{\mathbf{X}_i} \sigma_{\mathbf{X}_j}} $

In [7]:

def pearson_correlation(x,y):
    """ x,y denoted the signal_x and signal_y following the equation """
    cov = np.cov(x, y)
    # print(cov)
    # [[ 8806859.74527069  8007149.0906219 ] ==> [[cov_xx, cov_xy]
    # [ 8007149.0906219  10396797.72458848]]      [cov_yx, cov_yy]]
    cov_xy = cov[0,1] # or cov[1,0]
    cov_xx = cov[0,0]
    cov_yy = cov[1,1]
    corr = cov_xy / ( cov_xx**0.5 * cov_yy**0.5  )
    return corr

def _cal(p_id, partial_data):
    # print(f"p_id:{p_id} - data to run {partial_data.shape}")
    from itertools import combinations
    pcc = []
    for index in range(partial_data.shape[0]):
        pcc_epoch = []
        for comb in combinations(list(range(partial_data.shape[1])), 2):
            pcc_ab = pearson_correlation(partial_data[index, comb[0], :], partial_data[index, comb[1], :]   )
            pcc_epoch.append(pcc_ab)
        pcc_epoch = np.hstack(pcc_epoch)
        pcc.append(pcc_epoch)
    pcc = np.vstack(pcc)
    return pcc

def calculate_pcc(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(_cal, [p_id, data[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    except Exception as e:
        print(e)
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)

In [8]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    data_csd = mne.preprocessing.compute_current_source_density(data)
    pcc = calculate_pcc(data_csd.get_data())
    _,acc,cross,train_report = train_model(pcc, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pcc, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.6|10-CV=0.56|STD=0.1562|Time spend=1.3059582710266113
	VALENCE-s01|Acc=0.525|10-CV=0.44|STD=0.11136|Time spend=1.3651525974273682
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.6|10-CV=0.51|STD=0.15133|Time spend=1.8193936347961426
	VALENCE-s02|Acc=0.55|10-CV=0.47|STD=0.1005|Time spend=1.8768584728240967
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=1.3508203029632568
	VALENCE-s03|Acc=0.55|10-CV=0.41|STD=0.12207|Time spend=1.4137225151062012
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1

In [9]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.6|10-CV=0.56|STD=0.1562|Time spend=1.3059582710266113
-----WARNING: Model s01 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	VALENCE-s01|Acc=0.525|10-CV=0.44|STD=0.11136|Time spend=1.3651525974273682
-----WARNING: Model s01 failed to learn: sum(ans)=0.0 sum(y)=19.0 len(y)=40
	AROUSAL-s02|Acc=0.6|10-CV=0.51|STD=0.15133|Time spend=1.8193936347961426
-----WARNING: Model s02 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	VALENCE-s02|Acc=0.55|10-CV=0.47|STD=0.1005|Time spend=1.8768584728240967
-----WARNING: Model s02 failed to learn: sum(ans)=40.0 sum(y)=22.0 len(y)=40
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=1.3508203029632568
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=8.0 len(y)=40
	VALENCE-s03|Acc=0.55|10-CV=0.41|STD=0.12207|Time spend=1.4137225151062012
-----WARNING: Model s03 failed to learn: sum(ans)=40.0 sum(y)=22.0 len(y)=40
	AROUSAL-s04|Acc=0.6|10-CV=0.54|STD=0.22|Time spend=1.291790246963501
-----WARNING: Model s04 f

### 4.2 $ \text{PCC}_{freq} (i,j) = \frac{\text{Cov}[\hat{\mathbf{X}}_i, \hat{\mathbf{X}}_j]}{\sigma_{\hat{\mathbf{X}}_i} \sigma_{\hat{\mathbf{X}}_j}} $

In [10]:
def calculate_fft(signal, sfreq):
    """ signal: can be 1D array of (n_sample,) or 2D array of (n_signal, n_sample)  """
    number_sample = signal.shape[-1]

    # the result will be a complex number. We can obtain the magnitude using `absolute`
    magnitude = np.abs(np.fft.fft(signal))
    # scale the result
    magnitude = magnitude / (number_sample/2)
    # Selecting the range
    magnitude = magnitude.T[:number_sample//2].T
    freq_range = np.fft.fftfreq(number_sample, d=1/sfreq)[:number_sample//2]

    return magnitude, freq_range

In [11]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)
    magnitude, freq_range = calculate_fft(data_csd.get_data(), 128)
    pcc = calculate_pcc(magnitude)
    _,acc,cross,train_report = train_model(pcc, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pcc, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.725|10-CV=0.57|STD=0.16763|Time spend=1.0277223587036133
	VALENCE-s01|Acc=0.6|10-CV=0.38|STD=0.08718|Time spend=1.0744471549987793
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.6|10-CV=0.52|STD=0.13266|Time spend=0.8544299602508545
	VALENCE-s02|Acc=0.725|10-CV=0.53|STD=0.14177|Time spend=0.9031012058258057
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=0.8161563873291016
	VALENCE-s03|Acc=0.55|10-CV=0.42|STD=0.13266|Time spend=0.8696506023406982
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 1

In [12]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.725|10-CV=0.57|STD=0.16763|Time spend=1.0277223587036133
	VALENCE-s01|Acc=0.6|10-CV=0.38|STD=0.08718|Time spend=1.0744471549987793
	AROUSAL-s02|Acc=0.6|10-CV=0.52|STD=0.13266|Time spend=0.8544299602508545
-----WARNING: Model s02 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	VALENCE-s02|Acc=0.725|10-CV=0.53|STD=0.14177|Time spend=0.9031012058258057
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=0.8161563873291016
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=8.0 len(y)=40
	VALENCE-s03|Acc=0.55|10-CV=0.42|STD=0.13266|Time spend=0.8696506023406982
-----WARNING: Model s03 failed to learn: sum(ans)=40.0 sum(y)=22.0 len(y)=40
	AROUSAL-s04|Acc=0.6|10-CV=0.55|STD=0.19621|Time spend=0.8177976608276367
-----WARNING: Model s04 failed to learn: sum(ans)=0.0 sum(y)=16.0 len(y)=40
	VALENCE-s04|Acc=0.6|10-CV=0.49|STD=0.18682|Time spend=0.864173173904419
-----WARNING: Model s04 failed to learn: sum(ans)=0.0 sum(y)=16.0 len(y)=40
	AROUSAL-s05|Acc=0.575|

### 4.3 $ \text{PLV}(j,k) = \frac{1}{T} | \Sigma^{T}_{t=1} e^{i(\phi^{t}_{j} - \phi^{t}_{k})}   | $

In [13]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
def calculate_stft(signals, sfreq):
    from scipy import signal
    f_range, t_range, Z = signal.stft(signals, sfreq, nperseg=sfreq//10, nfft=sfreq)
    magnitude = np.abs(Z) 
    phase = np.angle(Z)
    return magnitude, phase, f_range, t_range

def PLV_stft(p_id, phase):
    # (32, 65, 12) => (channels, freqs, times)
    # print(f"p_id:{p_id} - data to run {phase.shape}")
    print(phase.shape)
    from itertools import combinations
    plv = []
    # count = 0
    for index in range(phase.shape[0]):
        plv_epoch = []
        for comb in combinations(list(range(phase.shape[1])), 2):
            # shape = (65,12)
            phase_a, phase_b = phase[index, comb[0]], phase[index, comb[1]]
            phase_diff = phase_a - phase_b
            # sum along the time size
            plv_ab = np.abs(np.average(np.exp(complex(0,1) * phase_diff), axis=1))
            plv_epoch.append(plv_ab)
        plv_epoch = np.vstack(plv_epoch)
        # print(plv_epoch.shape) => (300, 32, 65, 23)
        plv_epoch_5 = np.concatenate([ plv_epoch[:,0:4].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,4:8].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,8:12].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,12:30].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,30:65].mean(axis=1).reshape(-1,1)], axis=0)
        plv.append(np.expand_dims(plv_epoch_5, axis=0))
        # count += 1
        # if(count == 3): break
    # shape (496, 65)
    # 496 is number of pairs that is not duplicate
    # 65 is number of phase of frequencies
    plv = np.vstack( plv )
    return plv.squeeze()

def calculate_plv(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn, n_freqs ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        _, phase, _, _ = calculate_stft(data, 128)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(PLV_stft, [p_id, phase[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)


In [14]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)

    plv = calculate_plv(data_csd.get_data(), n_jobs=8)

    _,acc,cross,train_report = train_model(plv, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(plv, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
========= close ========
	AROUSAL-s01|Acc=0.6|10-CV=0.56|STD=0.1562|Time spend=23.686774492263794
	VALENCE-s01|Acc=0.525|10-CV=0.47|STD=0.12689|Time spend=23.933307886123657
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
(5, 32, 65, 1281)
========= close ========
	AROUSAL-s02|Acc=0.6|10-CV=0.49|STD=0.09434|Time spend=21.165319442749023
	VALENCE-s02|Acc=0.775|10-CV=0.68|STD=0.11662|Time spend=21.243257999420166
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin de

In [15]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.6|10-CV=0.56|STD=0.1562|Time spend=23.686774492263794
-----WARNING: Model s01 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	VALENCE-s01|Acc=0.525|10-CV=0.47|STD=0.12689|Time spend=23.933307886123657
-----WARNING: Model s01 failed to learn: sum(ans)=0.0 sum(y)=19.0 len(y)=40
	AROUSAL-s02|Acc=0.6|10-CV=0.49|STD=0.09434|Time spend=21.165319442749023
-----WARNING: Model s02 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	VALENCE-s02|Acc=0.775|10-CV=0.68|STD=0.11662|Time spend=21.243257999420166
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=21.91300058364868
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=8.0 len(y)=40
	VALENCE-s03|Acc=0.55|10-CV=0.5|STD=0.20976|Time spend=22.009908199310303
-----WARNING: Model s03 failed to learn: sum(ans)=40.0 sum(y)=22.0 len(y)=40
	AROUSAL-s04|Acc=0.6|10-CV=0.55|STD=0.19621|Time spend=21.160433530807495
-----WARNING: Model s04 failed to learn: sum(ans)=0.0 sum(y)=16.0 len(y)=40
	VALENCE-s04|Acc=0.6|1

### 4.4 $ \text{PLI}(j,k) =  \frac{1}{T} | \Sigma^{T}_{t=1} \text{sign}(Im[e^{i (\phi^{t}_{j} - \phi^{t}_{k})}])  | $

In [16]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
def calculate_stft(signals, sfreq):
    from scipy import signal
    f_range, t_range, Z = signal.stft(signals, sfreq, nperseg=sfreq//10, nfft=sfreq)
    magnitude = np.abs(Z) 
    phase = np.angle(Z)
    return magnitude, phase, f_range, t_range

def PLI_stft(p_id, phase):
    # (32, 65, 12) => (channels, freqs, times)
    # print(f"p_id:{p_id} - data to run {phase.shape}")
    from itertools import combinations
    pli = []
    # count = 0
    for index in range(phase.shape[0]):
        pli_epoch = []
        for comb in combinations(list(range(phase.shape[1])), 2):
            # shape = (65,12)
            phase_a, phase_b = phase[index, comb[0]], phase[index, comb[1]]
            phase_diff = phase_a - phase_b
            # sum along the time size
            pli_ab = np.abs(np.average(   np.sign(np.imag(   np.exp(complex(0,1) * phase_diff) ))  , axis=1))
            pli_epoch.append(pli_ab)
        pli_epoch = np.vstack(pli_epoch)
        pli_epoch_5 = np.concatenate([ pli_epoch[:,0:4].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,4:8].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,8:12].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,12:30].mean(axis=1).reshape(-1,1),
                                        pli_epoch[:,30:65].mean(axis=1).reshape(-1,1)], axis=0)
        pli.append(np.expand_dims(pli_epoch_5, axis=0))
        # count += 1
        # if(count == 3): break
    # shape (496, 65)
    # 496 is number of pairs that is not duplicate
    # 65 is number of phase of frequencies
    pli = np.vstack( pli )
    return pli.squeeze()

def calculate_pli(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn, n_freqs ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        _, phase, _, _ = calculate_stft(data, 128)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(PLI_stft, [p_id, phase[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)


In [17]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)

    pli = calculate_pli(data_csd.get_data(), n_jobs=8)

    _,acc,cross,train_report = train_model(pli, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pli, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=1.0|10-CV=0.51|STD=0.14457|Time spend=20.41959500312805
	VALENCE-s01|Acc=1.0|10-CV=0.46|STD=0.08|Time spend=20.49126625061035
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.875|10-CV=0.62|STD=0.1249|Time spend=18.83008646965027
	VALENCE-s02|Acc=0.775|10-CV=0.57|STD=0.11874|Time spend=18.907878160476685
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.85|10-CV=0.73|STD=0.15524|Time spend=19.465853214263916
	VALENCE-s03|Acc=1.0|10-CV=0.45|STD=0.13601|Time spend=19.628947257995605
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.

In [18]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=1.0|10-CV=0.51|STD=0.14457|Time spend=20.41959500312805
	VALENCE-s01|Acc=1.0|10-CV=0.46|STD=0.08|Time spend=20.49126625061035
	AROUSAL-s02|Acc=0.875|10-CV=0.62|STD=0.1249|Time spend=18.83008646965027
	VALENCE-s02|Acc=0.775|10-CV=0.57|STD=0.11874|Time spend=18.907878160476685
	AROUSAL-s03|Acc=0.85|10-CV=0.73|STD=0.15524|Time spend=19.465853214263916
	VALENCE-s03|Acc=1.0|10-CV=0.45|STD=0.13601|Time spend=19.628947257995605
	AROUSAL-s04|Acc=0.925|10-CV=0.53|STD=0.17916|Time spend=19.976188898086548
	VALENCE-s04|Acc=0.9|10-CV=0.5|STD=0.16125|Time spend=20.071115255355835
	AROUSAL-s05|Acc=0.825|10-CV=0.36|STD=0.08|Time spend=19.55043864250183
	VALENCE-s05|Acc=0.85|10-CV=0.78|STD=0.1249|Time spend=19.674164056777954
	AROUSAL-s06|Acc=0.975|10-CV=0.53|STD=0.14177|Time spend=19.654197692871094
	VALENCE-s06|Acc=0.95|10-CV=0.72|STD=0.07483|Time spend=19.835190773010254
	AROUSAL-s07|Acc=1.0|10-CV=0.65|STD=0.14318|Time spend=19.44927453994751
	VALENCE-s07|Acc=0.975|10-CV=0.69|STD=0

## 5. CSP

In [19]:
from components.CSP import CSP
from mne_features.feature_extraction import FeatureExtractor

bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

In [20]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    labels = labels.squeeze()
    start = time.time()
    filters = CSP(data[labels==0], data[labels==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[labels[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape[1] == 160
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


-----WARNING: Model s01 failed to learn: sum(ans)=40.0 sum(y)=24.0 len(y)=40
	AROUSAL-s01|Acc=0.6|10-CV=0.56|STD=0.1562|Time spend=0.8535566329956055


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s02|Acc=1.0|10-CV=1.0|STD=0.0|Time spend=0.7488725185394287


...

	AROUSAL-s30|Acc=0.9|10-CV=0.82|STD=0.1249|Time spend=0.5267007350921631


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s31|Acc=0.75|10-CV=0.56|STD=0.201|Time spend=0.5009779930114746


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


-----WARNING: Model s32 failed to learn: sum(ans)=40.0 sum(y)=27.0 len(y)=40
	AROUSAL-s32|Acc=0.675|10-CV=0.68|STD=0.2358|Time spend=0.5244612693786621
AROUSAL|Acc=0.8515625|10-CV=0.8184374999999999|STD=0.07952986706277952


In [21]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    labels = labels.squeeze()
    start = time.time()
    filters = CSP(data[labels==0], data[labels==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[labels[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape[1] == 160
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s01|Acc=0.975|10-CV=0.9|STD=0.1|Time spend=0.5669615268707275


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


-----WARNING: Model s02 failed to learn: sum(ans)=40.0 sum(y)=22.0 len(y)=40
	VALENCE-s02|Acc=0.55|10-CV=0.4|STD=0.1|Time spend=0.5128936767578125


...

	VALENCE-s30|Acc=0.925|10-CV=0.96|STD=0.06633|Time spend=0.5015664100646973


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s31|Acc=1.0|10-CV=1.0|STD=0.0|Time spend=0.5103473663330078


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s32|Acc=0.95|10-CV=0.95|STD=0.06708|Time spend=0.5320570468902588
VALENCE|Acc=0.8765625|10-CV=0.841875|STD=0.06790463672431077


In [22]:
accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)

print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

TOTAL|Acc=0.8640625000000001|10-CV=0.83015625|STD=0.07371725189354518
